In [1]:
# disable future warning
import warnings
warnings.simplefilter('ignore')

# other imports
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from meteostat import Stations, Daily
import geopandas as gp
import pandas as pd
from shapely.geometry import Point
from datetime import date, datetime
from tqdm import tqdm

In [8]:
# setup cache
Stations.cache_dir = "./prod/.meteostat/cache"

query = Stations()
query.bounds("DE") #TODO: increase region size to make border cases more accurate
stations = query.fetch()
stations.reset_index(inplace=True)
stations.head()

,id,name,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end
country,,,,,,,,,,,,,,,
AE,15,15,14,8,15,15,15,15,15,15,15,12,12,8,8
AF,42,42,42,36,40,42,42,42,42,40,40,25,25,13,13
AG,3,3,3,3,2,3,3,3,3,2,2,1,1,1,1
AL,10,10,10,10,5,10,10,10,10,10,10,1,1,5,5
AM,41,41,40,39,3,41,41,41,41,16,16,39,39,39,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XK,2,2,1,2,2,2,2,2,2,2,2,2,2,0,0
YE,12,12,12,11,12,12,12,12,12,10,10,5,5,2,2
ZA,141,141,137,110,85,141,141,141,141,85,85,103,103,92,92


In [40]:
# remove unnecessary station data
cols = [
    'id',
    'latitude',
    'longitude',
    'elevation',
    'daily_start',
    'daily_end'
]
stations = stations[cols]

In [41]:
# read shape files
shape = [
    gp.read_file(f"./data/nuts5000/5000_NUTS{i}.shp").to_crs(epsg=4326)
    for i in range(1, 4)
]

# spacial join shape files. This is possible since nuts is hirarchical
nuts:gp.GeoDataFrame
nuts = shape[2].sjoin(shape[1], how="left", lsuffix='3', rsuffix='2', predicate="within")\
               .sjoin(shape[0], how="left", rsuffix='1', predicate="within")
nuts.rename({"NUTS_CODE":"NUTS_CODE_1",	"NUTS_NAME":"NUTS_NAME_1"}, inplace=True, axis=1)
nuts.drop("NUTS_LEVEL_2 NUTS_LEVEL_3 NUTS_LEVEL index_2 index_1".split(), inplace=True, axis=1)
nuts.sort_index(inplace=True)
nuts.head()

,NUTS_CODE_3,NUTS_NAME_3,geometry,NUTS_CODE_2,NUTS_NAME_2,NUTS_CODE_1,NUTS_NAME_1
0,DE111,"Stuttgart, Stadtkreis","POLYGON ((9.13452 48.85668, 9.14122 48.86183, ...",DE11,Stuttgart,DE1,Baden-Württemberg
1,DE112,Böblingen,"POLYGON ((8.96647 48.82980, 8.99216 48.83356, ...",DE11,Stuttgart,DE1,Baden-Württemberg
2,DE113,Esslingen,"POLYGON ((9.40973 48.53721, 9.39153 48.53014, ...",DE11,Stuttgart,DE1,Baden-Württemberg
3,DE114,Göppingen,"POLYGON ((9.91934 48.63977, 9.94730 48.63369, ...",DE11,Stuttgart,DE1,Baden-Württemberg
4,DE115,Ludwigsburg,"MULTIPOLYGON (((9.30157 48.95210, 9.31516 48.9...",DE11,Stuttgart,DE1,Baden-Württemberg


In [42]:
# create GeoDataFrame from stations
x, y = stations["longitude"], stations["latitude"]
stations["geometry"] = gp.GeoSeries(map(Point, zip(x, y)))
stations = gp.GeoDataFrame(stations)
# spactial join with nuts data
stations = stations.sjoin(nuts, how="inner", predicate="within")
stations.drop("index_right", axis=1, inplace=True)
stations.sort_index(inplace=True)
stations.head()

,id,latitude,longitude,elevation,daily_start,daily_end,geometry,NUTS_CODE_3,NUTS_NAME_3,NUTS_CODE_2,NUTS_NAME_2,NUTS_CODE_1,NUTS_NAME_1
1230,10015,54.1833,7.9000,4.0,1952-05-01,2023-01-04,POINT (7.90000 54.18330),DEF09,Pinneberg,DEF0,Schleswig-Holstein,DEF,Schleswig-Holstein
1231,10018,54.9167,8.3500,16.0,2009-02-24,2022-04-25,POINT (8.35000 54.91670),DEF07,Nordfriesland,DEF0,Schleswig-Holstein,DEF,Schleswig-Holstein
1232,10020,55.0167,8.4167,26.0,1931-01-01,2023-01-04,POINT (8.41670 55.01670),DEF07,Nordfriesland,DEF0,Schleswig-Holstein,DEF,Schleswig-Holstein
1233,10022,54.8000,8.9500,7.0,1973-01-01,2023-01-04,POINT (8.95000 54.80000),DEF07,Nordfriesland,DEF0,Schleswig-Holstein,DEF,Schleswig-Holstein
1234,10026,54.5167,9.1500,28.0,1891-01-01,1974-06-30,POINT (9.15000 54.51670),DEF07,Nordfriesland,DEF0,Schleswig-Holstein,DEF,Schleswig-Holstein


In [43]:
# save to database
stations.to_file("prod/stationinfo.geojson", driver="GeoJSON")